In [ ]:
import os
import shutil
import random
from pathlib import Path

base_dir = Path("dataset/Data")
train_source = base_dir / "Train_Data"
test_source = base_dir / "Test_Data"

output_dir = Path("processed_dataset")

split_ratio = 0.8 

classes = ["Fire", "Non_Fire"]

def prepare_data():
    for split in ["train", "val", "test"]:
        for class_name in classes:
            (output_dir / split / class_name).mkdir(parents=True, exist_ok=True)

    print("Đang xử lý dữ liệu...")

    for class_name in classes:
        src_path = train_source / class_name
        files = [f for f in os.listdir(src_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        random.shuffle(files)
        
        split_idx = int(len(files) * split_ratio)
        train_files = files[:split_idx]
        val_files = files[split_idx:]
        
        for f in train_files:
            shutil.copy(src_path / f, output_dir / "train" / class_name / f)
        for f in val_files:
            shutil.copy(src_path / f, output_dir / "val" / class_name / f)
            
        print(f"Class '{class_name}': {len(train_files)} ảnh vào Train, {len(val_files)} ảnh vào Val.")

        src_test_path = test_source / class_name
        test_files = [f for f in os.listdir(src_test_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        for f in test_files:
            shutil.copy(src_test_path / f, output_dir / "test" / class_name / f)
        print(f"Class '{class_name}': Đã copy {len(test_files)} ảnh vào Test.")

    print(f"\nĐã chuẩn bị xong dữ liệu tại thư mục: {output_dir.absolute()}")

prepare_data()

Đang xử lý dữ liệu...
Class 'Fire': 2000 ảnh vào Train, 500 ảnh vào Val.
Class 'Fire': Đã copy 25 ảnh vào Test.
Class 'Non_Fire': 2000 ảnh vào Train, 500 ảnh vào Val.
Class 'Non_Fire': Đã copy 25 ảnh vào Test.

Đã chuẩn bị xong dữ liệu tại thư mục: d:\FSB\SEM1\IVP501\code\project\processed_dataset


In [2]:
from ultralytics import YOLO

model = YOLO("yolo11n-cls.pt") 
results = model.train(
    data="processed_dataset", 
    epochs=30,               
    imgsz=224,                
    batch=32,                 
    project="MyFireProject", 
    name="yolo11n_fire_run",
    close_mosaic=0,
    amp=True
)


New https://pypi.org/project/ultralytics/8.3.239 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.235  Python-3.11.14 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=0, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=processed_dataset, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_fire_run5, nb

In [5]:
results = model.val(data="processed_dataset", split="test")

Ultralytics 8.3.235  Python-3.11.14 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
YOLO11n-cls summary (fused): 47 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
train: D:\FSB\SEM1\IVP501\code\project\processed_dataset\train... found 4000 images in 2 classes  
val: D:\FSB\SEM1\IVP501\code\project\processed_dataset\val... found 1000 images in 2 classes  
test: D:\FSB\SEM1\IVP501\code\project\processed_dataset\test... found 50 images in 2 classes  
test: Fast image access  (ping: 0.10.0 ms, read: 501.6416.5 MB/s, size: 87.4 KB)
test: Scanning D:\FSB\SEM1\IVP501\code\project\processed_dataset\test... 50 images, 0 corrupt: 100% ━━━━━━━━━━━━ 50/50 776.9it/s 0.1s
test: New cache created: D:\FSB\SEM1\IVP501\code\project\processed_dataset\test.cache
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 4/4 2.2s/it 9.0s0.3s6
                   all       0.98          1
Speed: 0.5ms preprocess, 10.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Res